In [1]:
import requests
import pandas as pd
from riotwatcher import LolWatcher, ApiError
import mysql.connector
import time
import ast

In [2]:
api_key = 'RGAPI-da2b720d-229b-4885-9826-f62fd14b2801'
watcher = LolWatcher(api_key)
na = 'na1'

In [3]:
database_name = 'league-database'
username = 'admin'
password = 'CowDog019283'
endpoint = 'league-database.cgdkdqtwgxcv.us-east-2.rds.amazonaws.com'
port = 3306
db_name = 'leaguedb'

In [4]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()
query = "SELECT summonerid FROM summoners"
cursor.execute(query)

ids = []
for id in cursor:
    ids.append(id[0])

cursor.close()
connection.close()

In [5]:
len(ids)

9306

In [6]:
matches = []
for id in ids[:1500]:
    if id != None:
        try:
            matches.append(watcher.match.matchlist_by_puuid(na, id)[:10])
        except ApiError as err:
            if err.response.status_code == 429:
                print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
                print('this retry-after is handled by default by the RiotWatcher library')
                print('future requests wait until the retry-after time passes')
            elif err.response.status_code == 404:
                print('Not Found.')
            elif err.response.status_code == 403:
                print('Forbidden')
            else:
                raise

In [7]:
for id in ids[1500:3000]:
    if id != None:
        try:
            matches.append(watcher.match.matchlist_by_puuid(na, id)[:10])
        except ApiError as err:
            if err.response.status_code == 429:
                print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
                print('this retry-after is handled by default by the RiotWatcher library')
                print('future requests wait until the retry-after time passes')
            elif err.response.status_code == 404:
                print('Not Found.')
            elif err.response.status_code == 403:
                print('Forbidden')
            else:
                raise

In [13]:
for id in ids[5400:6000]:
    if id != None:
        try:
            matches.append(watcher.match.matchlist_by_puuid(na, id)[:10])
        except ApiError as err:
            if err.response.status_code == 429:
                print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
                print('this retry-after is handled by default by the RiotWatcher library')
                print('future requests wait until the retry-after time passes')
            elif err.response.status_code == 404:
                print('Not Found.')
            elif err.response.status_code == 403:
                print('Forbidden')
            else:
                raise

In [14]:
for id in ids[6000:7500]:
    if id != None:
        try:
            matches.append(watcher.match.matchlist_by_puuid(na, id)[:10])
        except ApiError as err:
            if err.response.status_code == 429:
                print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
                print('this retry-after is handled by default by the RiotWatcher library')
                print('future requests wait until the retry-after time passes')
            elif err.response.status_code == 404:
                print('Not Found.')
            elif err.response.status_code == 403:
                print('Forbidden')
            else:
                raise

In [19]:
# Definitely could have made a function for this, but I just turned off my brain and did this
for id in ids[7500:]:
    if id != None:
        try:
            matches.append(watcher.match.matchlist_by_puuid(na, id)[:10])
        except ApiError as err:
            if err.response.status_code == 429:
                print('We should retry in {} seconds.'.format(err.response.headers['Retry-After']))
                print('this retry-after is handled by default by the RiotWatcher library')
                print('future requests wait until the retry-after time passes')
            elif err.response.status_code == 404:
                print('Not Found.')
            elif err.response.status_code == 403:
                print('Forbidden')
            else:
                raise

In [ ]:
mat = []
for match in matches:
    for id in match:
        mat.append(id)

In [21]:
len(mat)

76407

In [4]:
match_df = pd.read_csv('match_ids.csv')
mat = match_df['ids']
len(mat)

76407

In [26]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()

query = '''CREATE TABLE matches(
        matchid VARCHAR(40),
        info TEXT
    )'''

cursor.execute(query)
cursor.close()
connection.close()

In [8]:
# These are the different types of function I have gone through, but the last one is best
def get_info(n, m):
    game_info = []
    for id in mat[n:m]:
        game_info.append(watcher.match.by_id(na, id))
    game_ids = [data['metadata']['matchId'] for data in game_info]
    df = pd.DataFrame({'id':game_ids, 'info': game_info})
    return df

In [9]:
def make_df(startn, inc, total):
    game_info = []
    endn = startn
    startn = startn - inc
    r = total % inc
    ran = total / inc
    dataf = pd.DataFrame(columns=['id', 'info'])
    try:
        if r == 0:
            for i in range(int(ran)):
                endn += inc
                startn += inc
                print(startn, endn)
                for id in mat[startn:endn]:
                    game_info.append(watcher.match.by_id(na, id))
                    time.sleep(1)
                time.sleep(45)
            game_ids = [data['metadata']['matchId'] for data in game_info]
            dataf['id'] = game_ids
            dataf['info'] = game_info 
        else:
            print('Total number divided by the increment has to be integer, not float')
    except ApiError as err:
        if err.response.status_code == 503:
            game_ids = [data['metadata']['matchId'] for data in game_info]
            dataf['id'] = game_ids
            dataf['info'] = game_info 
        else:
            raise
    return dataf

In [5]:
# This function worked best because even when I got an error, 
# my data saved and it would time out most of the time so all of the data I wanted
# This is why I used a recursive function
def force(start, end):
    game_info = []
    force_df = pd.DataFrame(columns=['id', 'info'])
    try:
        for id in mat[start:end]:
            game_info.append(watcher.match.by_id(na, id))
        game_ids = [data['metadata']['matchId'] for data in game_info]
        force_df['id'] = game_ids
        force_df['info'] = game_info 
    except ApiError as err:
        if err.response.status_code == 503:
            print('forcing')
            game_ids = [data['metadata']['matchId'] for data in game_info]
            force_df['id'] = game_ids
            force_df['info'] = game_info 
            force_df = pd.concat([force_df, force(start + force_df.shape[0], end)])
        else:
            print('forcing this way')
            game_ids = [data['metadata']['matchId'] for data in game_info]
            force_df['id'] = game_ids
            force_df['info'] = game_info 
            force_df = pd.concat([force_df, force(start + force_df.shape[0], end)])
    return force_df

In [7]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()
query = '''ALTER TABLE matches
        MODIFY COLUMN info LONGTEXT'''
cursor.execute(query)

In [ ]:
query = 'DESCRIBE matches'
cursor.execute(query)

In [ ]:
one = force(0, 10000)

In [ ]:
one.to_csv('one.csv')

In [ ]:
two = force(10000, 20000)

In [38]:
two.to_csv('two.csv')

In [ ]:
three = force(20000, 30000)

In [ ]:
three.to_csv('three.csv')

In [6]:
four = force(30000, 40000)

In [ ]:
four.to_csv('four.csv')

In [6]:
five = force(40000, 50000)

forcing
forcing


In [11]:
five

,Unnamed: 0,id,info
0,0.0,NA1_4363681956,"{'metadata': {'dataVersion': '2', 'matchId': '..."
1,1.0,NA1_4363608475,"{'metadata': {'dataVersion': '2', 'matchId': '..."
2,2.0,NA1_4363368464,"{'metadata': {'dataVersion': '2', 'matchId': '..."
3,3.0,NA1_4363371242,"{'metadata': {'dataVersion': '2', 'matchId': '..."
4,4.0,NA1_4363226492,"{'metadata': {'dataVersion': '2', 'matchId': '..."
...,...,...,...
6995,NaN,NA1_4365293664,"{'metadata': {'dataVersion': '2', 'matchId': '..."
6996,NaN,NA1_4365798276,"{'metadata': {'dataVersion': '2', 'matchId': '..."
6997,NaN,NA1_4364844883,"{'metadata': {'dataVersion': '2', 'matchId': '..."
6998,NaN,NA1_4364827418,"{'metadata': {'dataVersion': '2', 'matchId': '..."


In [12]:
five.to_csv('five.csv')

In [6]:
six = force(50000, 60000)

In [18]:
six.to_csv('six.csv')

In [19]:
seven = force(60000, 70000)

forcing
forcing


In [12]:
seven.to_csv('seven.csv')

In [17]:
last = force(70000, 76407)

forcing
forcing
forcing
forcing


In [18]:
last.to_csv('last.csv')

In [5]:
# connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
# cursor = connection.cursor()
# query = "DELETE FROM matches"
# cursor.execute(query)
# connection.commit()
# cursor.close()
# connection.close()

In [8]:
def dfToSql(df):
    connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
    cursor = connection.cursor()
    for row in df[['id', 'info']].itertuples():
        query = '''INSERT INTO matches(matchid, info)
                VALUES(%s, %s)'''

        add_id = (row[1], row[2])

        cursor.execute(query, add_id)
        connection.commit()

    cursor.close()
    connection.close()

In [6]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()
query = '''ALTER TABLE matches
            MODIFY COLUMN info MEDIUMTEXT'''
cursor.execute(query)

In [7]:
query = 'DESCRIBE matches'
cursor.execute(query)
cursor.fetchall()

[('matchid', b'varchar(40)', 'YES', bytearray(b''), None, ''),
 ('info', b'mediumtext', 'YES', bytearray(b''), None, '')]

In [9]:
one = pd.read_csv('one.csv')

In [10]:
two = pd.read_csv('two.csv')

In [11]:
three = pd.read_csv('three.csv')

In [12]:
four = pd.read_csv('four.csv')

In [13]:
five = pd.read_csv('five.csv')

In [14]:
six = pd.read_csv('six.csv')

In [15]:
seven = pd.read_csv('seven.csv')

In [16]:
rest = pd.read_csv('last.csv')

In [17]:
csvs = [one, two, three, four, five, six, seven, rest]

In [18]:
for csv in csvs:
    dfToSql(csv)

In [4]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()
query = 'DELETE FROM matches'
cursor.execute(query)
connection.commit()

In [19]:
# Taking all the csvs and putting it into one table in AWS
# I didn't have to make the csvs and I could've just done this with the variables, but since my computer could have crashed, I didn't take the risk
# The risk was that the code would take a while to run and I would lose all the data
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()
query = "SELECT * FROM matches"

pd.read_sql(query, connection)

C:\Users\ijosh\AppData\Roaming\Python\Python38\site-packages\pandas\io\sql.py:758: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


,matchid,info
0,NA1_4366795660,"{'metadata': {'dataVersion': '2', 'matchId': '..."
1,NA1_4366527467,"{'metadata': {'dataVersion': '2', 'matchId': '..."
2,NA1_4366533737,"{'metadata': {'dataVersion': '2', 'matchId': '..."
3,NA1_4365266709,"{'metadata': {'dataVersion': '2', 'matchId': '..."
4,NA1_4365293282,"{'metadata': {'dataVersion': '2', 'matchId': '..."
...,...,...
76402,NA1_4363734624,"{'metadata': {'dataVersion': '2', 'matchId': '..."
76403,NA1_4363741186,"{'metadata': {'dataVersion': '2', 'matchId': '..."
76404,NA1_4363687957,"{'metadata': {'dataVersion': '2', 'matchId': '..."
76405,NA1_4363672945,"{'metadata': {'dataVersion': '2', 'matchId': '..."


In [4]:
connection = mysql.connector.connect(host=endpoint, database='leaguedb', user=username, password=password)
cursor = connection.cursor()
query = '''ALTER TABLE matches RENAME TO matchIDs;'''
cursor.execute(query)